In [19]:
import random
import os
import numpy as np
import torch

def seed_everything(seed=1):
    """
    """
    # 1. Python & Numpy
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    # 2. PyTorch (CPU & GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    print(f"🔒 Locked Random Seed: {seed}")

seed_everything(10)

🔒 Locked Random Seed: 10


In [20]:
#import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from model import DragonNetBase, tarreg_loss, dragonnet_loss, EarlyStopper

In [21]:
#load data
df_women =pd.read_csv(r"C:\Users\Lenovo\Documents\Neu 2025-2026\Lab\Hillstrom-Women.csv")
df_women = df_women.drop(columns="Unnamed: 0")
print ("---------------------------")
print ("null count:")
print (df_women.isnull().sum())
print ("---------------------------")
print(df_women.dtypes)
print ("---------------------------")
print ("labels:")
print(df_women.columns.tolist())
print ("---------------------------")
print("data shape:")
print(df_women.shape)


---------------------------
null count:
recency            0
history_segment    0
history            0
mens               0
womens             0
zip_code           0
newbie             0
channel            0
visit              0
conversion         0
spend              0
treatment          0
dtype: int64
---------------------------
recency              int64
history_segment     object
history            float64
mens                 int64
womens               int64
zip_code            object
newbie               int64
channel             object
visit                int64
conversion           int64
spend              float64
treatment            int64
dtype: object
---------------------------
labels:
['recency', 'history_segment', 'history', 'mens', 'womens', 'zip_code', 'newbie', 'channel', 'visit', 'conversion', 'spend', 'treatment']
---------------------------
data shape:
(42693, 12)


In [22]:
#Hillstrom-men
#split num and cate
num_cols = df_women.select_dtypes(include=["float64", "int64"]).columns
cate_cols = ['zip_code', 'channel']
df_women["history_segment"] =df_women["history_segment"].map({
    "1) $0 - $100": '1', 
    "2) $100 - $200": 2, 
    "3) $200 - $350": "3",
    "4) $350 - $500": "4",
    "5) $500 - $750": "5",
    "6) $750 - $1,000": "6",
    "7) $1,000 +": "7"                         
})
#split x y t
y_women = df_women["spend"]
t_women = df_women["treatment"]
x_women = df_women.drop(columns=["spend", "treatment", "visit", "conversion"])

x_women_encode = pd.get_dummies(x_women, columns=cate_cols, drop_first=True)
#train test split
x_women_train, x_women_test_val,t_women_train, t_women_test_val, y_women_train, y_women_test_val = train_test_split(x_women_encode,t_women.values, y_women.values, test_size=0.4, random_state=42, stratify=t_women)
x_women_val, x_women_test, t_women_val, t_women_test, y_women_val, y_women_test = train_test_split(x_women_test_val, t_women_test_val, y_women_test_val, test_size= 0.75, random_state=42, stratify=t_women_test_val)

#scale
scaler = StandardScaler()
x_women_train = scaler.fit_transform(x_women_train)
x_women_val = scaler.transform(x_women_val)
x_women_test = scaler.transform(x_women_test)

In [23]:
#Transform to tensor
def to_tensor(df):
    return torch.tensor(df, dtype=torch.float32)

x_women_train_t = to_tensor(x_women_train)
x_women_val_t = to_tensor(x_women_val)
x_women_test_t = to_tensor(x_women_test)

y_women_train_t = to_tensor(y_women_train).unsqueeze(1)
y_women_val_t = to_tensor(y_women_val).unsqueeze(1)
y_women_test_t = to_tensor(y_women_test).unsqueeze(1)

t_women_train_t = to_tensor(t_women_train.astype(float)).unsqueeze(1)
t_women_val_t = to_tensor(t_women_val.astype(float)).unsqueeze(1)
t_women_test_t = to_tensor(t_women_test.astype(float)).unsqueeze(1)

#Data loader
train_dataset = TensorDataset(x_women_train_t, t_women_train_t, y_women_train_t)
val_dataset = TensorDataset(x_women_val_t, t_women_val_t, y_women_val_t)
test_dataset = TensorDataset(x_women_test_t, t_women_test_t, y_women_test_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print ("-------------------------------------------------------------")
print ("✅Completed tranform to tensor✅")
print (f"Shape of train: x={x_women_train_t.shape}; y ={y_women_train_t.shape}; t={t_women_train_t.shape}")
print (f"Shape of val: x={x_women_val_t.shape}; y={y_women_val_t.shape}; t={t_women_val_t.shape}")
print (f"Shape of test: x={x_women_test_t.shape}; y={y_women_test_t.shape}; t={t_women_test_t.shape}")



-------------------------------------------------------------
✅Completed tranform to tensor✅
Shape of train: x=torch.Size([25615, 10]); y =torch.Size([25615, 1]); t=torch.Size([25615, 1])
Shape of val: x=torch.Size([4269, 10]); y=torch.Size([4269, 1]); t=torch.Size([4269, 1])
Shape of test: x=torch.Size([12809, 10]); y=torch.Size([12809, 1]); t=torch.Size([12809, 1])


Evaluation metrics

In [24]:
from metrics import auuc, auqc, lift

Build Model

In [25]:
class DragonNetWrapper:
    def __init__(self, input_dim, epochs=50, lr = 0.001, alpha =1.0, beta =1.0):
        self.model = DragonNetBase(input_dim)
        self.epochs = epochs
        self.optim = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.alpha = alpha
        self.beta = beta
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.early_stopper = EarlyStopper(patience=10, min_delta=0)
        
    def fit (self, train_loader, val_loader):
        print (f"Begin training Dragonnet Baseline🔃🔃🔃 ") 
        early_stopper = EarlyStopper(patience=10, min_delta=0)
        for epoch in range(self.epochs):
            self.model.train()
            epoch_loss=0
            for x_batch , t_batch ,y_batch in train_loader:
                x_batch = x_batch.to(self.device)
                
                t_batch =t_batch.to(self.device) 
                y_batch = y_batch.to(self.device)
                
                self.optim.zero_grad()
                
                y0_pred, y1_pred, t_pred, eps = self.model(x_batch)
                
                loss = tarreg_loss(y_batch, t_batch, t_pred, y0_pred, y1_pred, eps, self.alpha, self.beta )
                
                loss.backward()
                self.optim.step()
                epoch_loss += loss.item()
            
            val_loss = self.validate(val_loader)
            
            if (epoch+1) % 5 == 0:
                print(f"Epoch {epoch+1} | Train Loss: {epoch_loss/len(train_loader):.4f} | VAL LOSS: {val_loss:.4f}")
            if self.early_stopper.early_stop(val_loss):
                print(f"⏹️ Dừng sớm tại epoch {epoch+1} vì Val Loss không giảm nữa.")
                break
    def validate(self, val_loader):
        self.model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, t, y in val_loader:
                X, t, y = X.to(self.device), t.to(self.device), y.to(self.device)
                y0, y1, t_p, eps = self.model(X)
                val_loss += tarreg_loss(y, t, t_p, y0, y1, eps, self.alpha, self.beta).item()
        return val_loss / len(val_loader)
                
    def predict(self, x):
        self.model.eval()
        with torch.no_grad():
            y0, y1, t_p, _  = self.model(x.to(self.device))
        return y0.cpu(), y1.cpu(), t_p.cpu()

In [26]:
dragonnet = DragonNetWrapper(input_dim=x_women_train_t.shape[1], epochs=50, alpha = 1.0, beta =1.0, lr=0.001)
dragonnet.fit(train_loader, val_loader)

print ("Complete training")

Begin training Dragonnet Baseline🔃🔃🔃 
Epoch 5 | Train Loss: 28141.4132 | VAL LOSS: 11458.2094
Epoch 10 | Train Loss: 28071.7639 | VAL LOSS: 11456.3883
Epoch 15 | Train Loss: 27993.4615 | VAL LOSS: 11466.1209
⏹️ Dừng sớm tại epoch 17 vì Val Loss không giảm nữa.
Complete training


In [27]:
print ("Evaluating baseline🔃🔃🔃")
y0_pred, y1_pred, _ = dragonnet.predict(x_women_test_t)

uplift_pred = (y1_pred - y0_pred).numpy().flatten()

y_true = y_women_test_t.numpy().flatten()
t_true = t_women_test_t.numpy().flatten()
auuc = auuc(y_true, t_true, uplift_pred, bins=100, plot=True)
auqc = auqc(y_true, t_true, uplift_pred, bins=100, plot=True)
lift = lift(y_true, t_true, uplift_pred, bins=100, plot=True)
print("-"*40)
print (auuc)
print (auqc)
print (lift)

Evaluating baseline🔃🔃🔃
Bucket nan: Empty group, skip


IndexError: list index out of range